In [1]:
import tkinter as tk
from tkinter import *
import tkinter.messagebox
import pyaudio
import wave
import os
import numpy as np
import math
import operator

In [2]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
WAVE_OUTPUT_FILENAME = "output/voice.wav"

In [3]:
p = pyaudio.PyAudio()
recording = 0
frames = []
index = 0

In [4]:
import pickle
with open("modelHMM.pkl", "rb") as file:
    models = pickle.load(file)

In [5]:
import functionMFCC as func

C:\Users\PhuongDong\miniconda3\envs\voice\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\PhuongDong\miniconda3\envs\voice\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [6]:
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

In [7]:
import soundfile as sf
import scipy as sp
import librosa

def trim_silence(y):
    y_trimmed, index = librosa.effects.trim(y, top_db=25, frame_length=25, hop_length=500) 
    trimmed_length = librosa.get_duration(y) - librosa.get_duration(y_trimmed)
    return y_trimmed, trimmed_length

def remove_noise():
    y,sr = librosa.load(os.path.join("output", "voice.wav"))
    y_reduced_median = sp.signal.medfilt(y,3) # lọc trung vị
    y_reduced_median, time_trimmed = trim_silence(y_reduced_median)
    sf.write(os.path.join("output", "voice_2.wav"),y_reduced_median, sr, 'PCM_24')
    print(f"Remove noise: {librosa.get_duration(y)} (s) -> {librosa.get_duration(y_reduced_median)} (s)")

In [8]:
text_predicted = {'cua': 'của', 'viet_nam': 'Việt Nam', 'khong': 'không', 'nguoi': 'người', 'nha': 'nhà'}
def predict_voice():
    remove_noise()
    inp = func.get_mfcc(os.path.join("output", "voice.wav"))
    score = {cname: model.score(inp, [len(inp)]) for cname, model in models.items()}
    print(score)
    text = max([(value, key) for key, value in score.items()])[1]
    outputBox.delete("1.0", END)
    outputBox.insert(END, text_predicted[text])

In [9]:
def start_record():
    recording = 1
    frames.clear()
    label['text'] = 'Đang ghi âm'
    stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
    while recording == 1:
        data = stream.read(CHUNK)
        frames.append(data)
        window.update()

In [10]:
def stop_record():
    recording = 0
    label['text'] = 'Dự đoán là'
    print('* stop recording')
    stream.stop_stream()
    stream.close()
    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    predict_voice()

In [11]:
def play_wav():
    p_read = pyaudio.PyAudio()
    wf_read = wave.open(os.path.join("output", "voice.wav"), 'rb')
    stream_read = p_read.open(format =
                            p_read.get_format_from_width(wf_read.getsampwidth()),
                            channels = wf_read.getnchannels(),
                            rate = wf_read.getframerate(),
                            output = True)
    data = wf_read.readframes(CHUNK)
    while data != '':
        # writing to the stream is what *actually* plays the sound.
        stream_read.write(data)
        data = wf_read.readframes(CHUNK)
        window.update()
    stream_read.close()    
    p_read.terminate()

In [12]:
window = tk.Tk()
window.geometry('500x200')
window.title('Predict voice')
button_start = tk.Button(window, text='Start', width=15, command=start_record)
button_start.place(x=70, y=50)
button_stop = tk.Button(window, text='Stop', width=15, command=stop_record)
button_stop.place(x=190, y=50)
button_play = tk.Button(window, text='Play', width=15, command=play_wav)
button_play.place(x=310, y=50)
label = tk.Label(window)
label.place(x=230, y=100)
outputBox = Text(window, width=30, height=1, bd=1)
outputBox.place(x=150, y=130)
window.mainloop()

* stop recording


Degenerate mixture covariance


Remove noise: 0.7430385487528345 (s) -> 0.49360544217687075 (s)
{'khong': -9238.935145680642, 'nha': -8613.339475989269, 'cua': -8452.746115012593, 'nguoi': -8611.251217690116, 'viet_nam': -9005.339966456744}


Degenerate mixture covariance


* stop recording
Remove noise: 0.6501587301587302 (s) -> 0.5215419501133787 (s)
{'khong': -7514.37455067848, 'nha': -7286.79453337105, 'cua': -7323.8960757387795, 'nguoi': -7643.278605564454, 'viet_nam': -7679.272988636395}


...

Degenerate mixture covariance


* stop recording
Remove noise: 0.7198185941043084 (s) -> 0.7198185941043084 (s)
{'khong': -8545.454668732695, 'nha': -8610.527743319159, 'cua': -8360.497376423118, 'nguoi': -8224.192147181484, 'viet_nam': -8569.790452201281}


Degenerate mixture covariance


* stop recording
Remove noise: 0.7662585034013606 (s) -> 0.6075283446712018 (s)
{'khong': -8888.472396007492, 'nha': -8507.600714178849, 'cua': -8622.04045222257, 'nguoi': -8997.046473421287, 'viet_nam': -8318.88180792611}


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\PhuongDong\miniconda3\envs\voice\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-9-91ae551fdff6>", line 13, in start_record
    window.update()
  File "C:\Users\PhuongDong\miniconda3\envs\voice\lib\tkinter\__init__.py", line 1177, in update
    self.tk.call('update')
_tkinter.TclError: can't invoke "update" command: application has been destroyed
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\PhuongDong\miniconda3\envs\voice\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-9-91ae551fdff6>", line 13, in start_record
    window.update()
  File "C:\Users\PhuongDong\miniconda3\envs\voice\lib\tkinter\__init__.py", line 1177, in update
    self.tk.call('update')
_tkinter.TclError: can't invoke "update" command: application has been destroyed
Exception in Tkinter callback
Trac